https://adventofcode.com/2023/day/14

In [225]:
with open("data/14.txt") as fh:
    puzzle = fh.read()

In [226]:
testdata = """\
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
"""

In [227]:
def parse_data(data):
    rounds = set()
    flats = set()
    for r, line in enumerate(data.splitlines()):
        for c, char in enumerate(line):
            pos = c - r * 1j
            if char == "O":
                rounds.add(pos)
            elif char == "#":
                flats.add(pos)
    return rounds, flats, r + 1, c + 1

In [228]:
def platform_to_string(rounds, flats, height, width):
    lines = []
    for r in range(height):
        line = []
        for c in range(width):
            pos = c - r * 1j
            line.append("O" if pos in rounds else "#" if pos in flats else ".")
        lines.append(" ".join(line))
    return "\n".join(lines)

In [230]:
def tilt_north(rounds, flats, height, width):
    for r, c in ((r, c) for r in range(1, height) for c in range(width)):
        pos = c - r * 1j
        above = pos + 1j
        if pos in rounds and above not in rounds and above not in flats:
            while True:
                nextabove = above + 1j
                if nextabove in rounds or nextabove in flats or nextabove.imag > 0:
                    break
                above = nextabove
            rounds.remove(pos)
            rounds.add(above)

In [233]:
def sum_rockloads(rounds, height):
    return sum(height + rock.imag for rock in rounds)

In [235]:
def total_load_on_north_support_beams(data):
    rounds, flats, height, width = parse_data(data)
    tilt_north(rounds, flats, height, width)
    return sum_rockloads(rounds, height)

In [236]:
total_load_on_north_support_beams(testdata)

136.0

In [237]:
total_load_on_north_support_beams(puzzle)

108840.0

### Part 2

In [263]:
def rotate_east(pts, height, width):
    newpts = set()
    for p in pts:
        c, r = int(p.real), -int(p.imag)
        r1 = c
        c1 = height - 1 - r
        newpts.add(c1 - r1 * 1j)
    return newpts, width, height


def spincycle(rounds, flats, height, width):
    for _ in range(4):
        tilt_north(rounds, flats, height, width)
        rounds, _, _ = rotate_east(rounds, height, width)
        flats, height, width = rotate_east(flats, height, width)
    return rounds, flats, height, width

In [264]:
def spincycle_cycle(rounds, flats, height, width):
    D = {}
    for i in range(1, 1_000):
        rounds, flats, height, width = spincycle(rounds, flats, height, width)
        frounds = frozenset(rounds)
        i0 = D.get(frounds)
        if i0 is not None:
            return i - i0, i0
        D[frounds] = i
    print("No cycle detected")
        

In [265]:
def extended_test(data, N):
    rounds, flats, height, width = parse_data(data)
    rounds0 = rounds.copy()
    cycle_length, offset = spincycle_cycle(rounds, flats, height, width)
    n = offset + (N - offset) % cycle_length
    rounds = rounds0
    for _ in range(n):
        rounds, flats, height, width = spincycle(rounds, flats, height, width)
    return sum_rockloads(rounds, height)
    

In [266]:
extended_test(testdata, 1_000_000_000)

64.0

In [267]:
extended_test(puzzle, 1_000_000_000)

103445.0